###  4.4.2 version made by (2 bytes)

#### 4.4.2.1 The upper byte indicates the compatibility of the file
        attribute information.  If the external file attributes 
        are compatible with MS-DOS and can be read by PKZIP for 
        DOS version 2.04g then this value will be zero.  If these 
        attributes are not compatible, then this value will 
        identify the host system on which the attributes are 
        compatible.  Software can use this information to determine
        the line record format for text files etc.  

#### 4.4.2.2 The current mappings are:

         0 - MS-DOS and OS/2 (FAT / VFAT / FAT32 file systems)
         1 - Amiga                     2 - OpenVMS
         3 - UNIX                      4 - VM/CMS
         5 - Atari ST                  6 - OS/2 H.P.F.S.
         7 - Macintosh                 8 - Z-System
         9 - CP/M                     10 - Windows NTFS
        11 - MVS (OS/390 - Z/OS)      12 - VSE
        13 - Acorn Risc               14 - VFAT
        15 - alternate MVS            16 - BeOS
        17 - Tandem                   18 - OS/400
        19 - OS X (Darwin)            20 thru 255 - unused

#### 4.4.2.3 The lower byte indicates the ZIP specification version 
        (the version of this document) supported by the software 
        used to encode the file.  The value/10 indicates the major 
        version number, and the value mod 10 is the minor version 
        number.  
        
### 4.4.5 compression method: (2 bytes)

        0 - The file is stored (no compression)
        1 - The file is Shrunk
        2 - The file is Reduced with compression factor 1
        3 - The file is Reduced with compression factor 2
        4 - The file is Reduced with compression factor 3
        5 - The file is Reduced with compression factor 4
        6 - The file is Imploded
        7 - Reserved for Tokenizing compression algorithm
        8 - The file is Deflated
        9 - Enhanced Deflating using Deflate64(tm)
       10 - PKWARE Data Compression Library Imploding (old IBM TERSE)
       11 - Reserved by PKWARE
       12 - File is compressed using BZIP2 algorithm
       13 - Reserved by PKWARE
       14 - LZMA (EFS)
       15 - Reserved by PKWARE
       16 - Reserved by PKWARE
       17 - Reserved by PKWARE
       18 - File is compressed using IBM TERSE (new)
       19 - IBM LZ77 z Architecture (PFS)
       97 - WavPack compressed data
       98 - PPMd version I, Rev 1

### 6.0  Traditional PKWARE Encryption

####    6.0.1 The following information discusses the decryption steps

    required to support traditional PKWARE encryption.  This
    form of encryption is considered weak by today's standards
    and its use is recommended only for situations with
    low security needs or for compatibility with older .ZIP 
    applications.

#### 6.1 Traditional PKWARE Decryption


    6.1.1 PKWARE is grateful to Mr. Roger Schlafly for his expert 
    contribution towards the development of PKWARE's traditional 
    encryption.

    6.1.2 PKZIP encrypts the compressed data stream.  Encrypted files 
    must be decrypted before they can be extracted to their original
    form.

    6.1.3 Each encrypted file has an extra 12 bytes stored at the start 
    of the data area defining the encryption header for that file.  The
    encryption header is originally set to random values, and then
    itself encrypted, using three, 32-bit keys.  The key values are
    initialized using the supplied encryption password.  After each byte
    is encrypted, the keys are then updated using pseudo-random number
    generation techniques in combination with the same CRC-32 algorithm
    used in PKZIP and described elsewhere in this document.

    6.1.4 The following are the basic steps required to decrypt a file:

    1) Initialize the three 32-bit keys with the password.
    2) Read and decrypt the 12-byte encryption header, further
       initializing the encryption keys.
    3) Read and decrypt the compressed data stream using the
       encryption keys.

    6.1.5 Initializing the encryption keys
        
    Key(0) <- 305419896
    Key(1) <- 591751049
    Key(2) <- 878082192

    loop for i <- 0 to length(password)-1
        update_keys(password(i))
    end loop

    Where update_keys() is defined as:

    update_keys(char):
      Key(0) <- crc32(key(0),char)
      Key(1) <- Key(1) + (Key(0) & 000000ffH)
      Key(1) <- Key(1) * 134775813 + 1
      Key(2) <- crc32(key(2),key(1) >> 24)
    end update_keys

    Where crc32(old_crc,char) is a routine that given a CRC value and a
    character, returns an updated CRC value after applying the CRC-32
    algorithm described elsewhere in this document.

    6.1.6 Decrypting the encryption header
        
    The purpose of this step is to further initialize the encryption
    keys, based on random data, to render a plaintext attack on the
    data ineffective.

    Read the 12-byte encryption header into Buffer, in locations
    Buffer(0) thru Buffer(11).

    loop for i <- 0 to 11
        C <- buffer(i) ^ decrypt_byte()
        update_keys(C)
        buffer(i) <- C
    end loop

    Where decrypt_byte() is defined as:

    unsigned char decrypt_byte()
        local unsigned short temp
        temp <- Key(2) | 2
        decrypt_byte <- (temp * (temp ^ 1)) >> 8
    end decrypt_byte

    After the header is decrypted,  the last 1 or 2 bytes in Buffer
    should be the high-order word/byte of the CRC for the file being
    decrypted, stored in Intel low-byte/high-byte order.  Versions of
    PKZIP prior to 2.0 used a 2 byte CRC check; a 1 byte CRC check is
    used on versions after 2.0.  This can be used to test if the password
    supplied is correct or not.

    6.1.7 Decrypting the compressed data stream
    
    The compressed data stream can be decrypted as follows:

    loop until done
        read a character into C
        Temp <- C ^ decrypt_byte()
        update_keys(temp)
        output Temp
    end loop


### Reference
* [ZIP File Format Specification](https://pkware.cachefly.net/webdocs/casestudies/APPNOTE.TXT)
* Wikipedia [ZIP](https://en.wikipedia.org/wiki/Zip_%28file_format%29)


In [9]:
File_Entry = [
    ("Signature", 4, "hex"), # 50 4b 03 04
    ("Minimum version needed", 2, "hex"),
    ("Flag", 2, "hex"),
    ("Compression Method", 2, "hex"),
    ("Last Modification Time", 2, "hex"),
    ("Last Modification Date", 2, "hex"),
    ("CRC", 4, "hex"),
    ("Compressed Size", 4, "integer"),
    ("Uncompressed Size", 4, "integer"),
    ("File Name Length", 2, "integer"),
    ("Extra Field Length", 2, "integer"),
    ("File Name", "File Name Length", "string"),
    ("Extra Field", "Extra Field Length", "hex"),
    ("File Data", "Compressed Size", "hex"),
]

Central_Directory_Record = [
    ("Signature", 4, "hex"), # 50 4b 01 02
    ("Version made by", 2, "hex"),
    ("Minimum version needed", 2, "hex"),
    ("Flag", 2, "hex"),
    ("Compression Method", 2, "hex"),
    ("Last Modification Time", 2, "hex"),
    ("Last Modification Date", 2, "hex"),
    ("CRC", 4, "hex"),
    ("Compressed Size", 4, "integer"),
    ("Uncompressed Size", 4, "integer"),
    ("File Name Length", 2, "integer"),
    ("Extra Field Length", 2, "integer"),
    ("Comment length", 2, "integer"),
    ("Disk number where file starts", 2, "integer"),
    ("Internal file attributes", 2, "hex"),
    ("External file attributes", 4, "hex"),
    ("Relative offset of local file header", 4, "integer"),
    ("File Name", "File Name Length", "string"),
    ("Extra Field", "Extra Field Length", "hex"),
    ("Comment", "Comment length", "string"),
]

End_Of_Central_Directory_Record = [
    ("Signature", 4, "hex"), # 50 4b 05 06
    ("Number of this disk", 2, "integer"),
    ("Disk where central directory starts", 2, "integer"),
    ("Number of central directory records on this disk", 2, "integer"),
    ("Total number of central directory records", 2, "integer"),
    ("Size of central directory", 4, "integer"),
    ("Offset of start of central directory", 4, "integer"),
    ("Comment length", 2, "integer"),
    ("Comment", "Comment length", "string"),
]

FORM = {
    "File_Entry": File_Entry,
    "Central_Directory_Record": Central_Directory_Record,
    "End_Of_Central_Directory_Record": End_Of_Central_Directory_Record,
}

In [22]:
def split(data):
    rexp = b'PK[\x01-\x08]{2}'
    signature = {
        b'PK\x01\x02': "Central_Directory_Record",
        b'PK\x03\x04': "File_Entry",
        b'PK\x05\x06': "End_Of_Central_Directory_Record",
    }
    temp = searchall(rexp, data)
    result = list()
    n = len(temp)
    for i in range(0, n):
        start = temp[i].start()
        form = signature[temp[i][0]]
        end = temp[i+1].start() if i+1 < n else len(data)
        result.append((start, end, form))
    return result

def display(result):
    for item in result:
        if type(item[2]) == list:
            print(item[0])
            for subitem in item[2]:
                if type(item[2]) == str and len(item[2]) > 128:
                    item = (item[0], item[1], item[2][:64] + "...")
                print("\t{0:16}\t{2}".format(*subitem))
        else:
            if item[0] != "unused":
                if type(item[2]) == str and len(item[2]) > 128:
                    item = (item[0], item[1], item[2][:64] + "...")
                print("{0:16}\t{2}".format(*item))

In [3]:
%run ../SourceCode/Parser.ipynb
%run ../SourceCode/FileProcessor.ipynb
%run ../SourceCode/Utilities.ipynb

In [4]:
folder = "../SampleFile/"
result_listdir = listdir(folder)

List of files in the directory ../SampleFile/
  0  CERT.RSA                        	        1714 Bytes
  1  drum.ani                        	        3240 Bytes
  2  first.txt                       	           5 Bytes
  3  flame.avi                       	      289280 Bytes
  4  FLOPPY.img                      	     1474560 Bytes
  5  MBR.img                         	         512 Bytes
  6  regedt32.exe                    	       10240 Bytes
  7  sda1_block_0.img                	        4096 Bytes
  8  sda1_block_1.img                	        4096 Bytes
  9  sda1_block_2.img                	        4096 Bytes
 10  sda1_block_3.img                	        4096 Bytes
 11  sda_1.img                       	        8192 Bytes
 12  sda_2.img                       	        1024 Bytes
 13  sda_5.img                       	        1024 Bytes
 14  sda_mbr.img                     	        1024 Bytes
 15  second.txt                      	           6 Bytes
 16  test.ani                        	    

In [32]:
path = result_listdir[23].path
test = FileProcessor(path)

load: ../SampleFile/test_enc.zip 267 Bytes
display: 1 / 1
000000000	50 4b 03 04 14 00 01 00 08 00 a1 65 24 4d 57 ee 	PK.........e$MW.
000000010	71 92 13 00 00 00 05 00 00 00 09 00 08 00 66 69 	q.............fi
000000020	72 73 74 2e 74 78 74 7a e5 04 00 b5 03 00 00 b9 	rst.txtz........
000000030	82 0d e5 c9 e7 f0 04 a2 5e 2a 62 03 af bf dc 52 	........^*b....R
000000040	b7 44 50 4b 03 04 14 00 01 00 08 00 ab 65 24 4d 	.DPK.........e$M
000000050	69 11 1f b6 14 00 00 00 06 00 00 00 0a 00 00 00 	i...............
000000060	73 65 63 6f 6e 64 2e 74 78 74 b9 82 0d e5 c9 e7 	second.txt......
000000070	f0 04 a2 5e 44 cd d6 69 7f 1a 90 bf 54 f5 50 4b 	...^D..i....T.PK
000000080	01 02 14 00 14 00 01 00 08 00 a1 65 24 4d 57 ee 	...........e$MW.
000000090	71 92 13 00 00 00 05 00 00 00 09 00 08 00 00 00 	q...............
0000000a0	00 00 01 00 20 00 00 00 00 00 00 00 66 69 72 73 	............firs
0000000b0	74 2e 74 78 74 7a e5 04 00 b5 03 00 00 50 4b 01 	t.txtz.......PK.
0000000c0	02 14 00 14 00 01 00

In [33]:
result_split = split(test.in_data)
for (start, end, form) in result_split:
    data = test.in_data[start:end]
    result = parser(data, form)
    display(result)
    print("\n")

1: 00000000 b'PK\x03\x04'
2: 00000042 b'PK\x03\x04'
3: 0000007e b'PK\x01\x02'
4: 000000bd b'PK\x01\x02'
5: 000000f5 b'PK\x05\x06'
Signature       	504b0304
Minimum version needed	1400
Flag            	0100
Compression Method	0800
Last Modification Time	a165
Last Modification Date	244d
CRC             	57ee7192
Compressed Size 	19
Uncompressed Size	5
File Name Length	9
Extra Field Length	8
File Name       	first.txt
Extra Field     	7ae50400b5030000
File Data       	b9820de5c9e7f004a25e2a6203afbfdc52b744


Signature       	504b0304
Minimum version needed	1400
Flag            	0100
Compression Method	0800
Last Modification Time	ab65
Last Modification Date	244d
CRC             	69111fb6
Compressed Size 	20
Uncompressed Size	6
File Name Length	10
Extra Field Length	0
File Name       	second.txt
Extra Field     	None
File Data       	b9820de5c9e7f004a25e44cdd6697f1a90bf54f5


Signature       	504b0102
Version made by 	1400
Minimum version needed	1400
Flag            	0100
Compression Method